In [1]:
%reset

In [2]:
from functools import cache
from itertools import product 

@cache
def get_factorizations(n):
    factorizations = set()
    factorizations.add((n,))

    divisor = 2
    while divisor**2 <= n:
        quotient, remainder = divmod(n, divisor)
        if remainder == 0:
            for a, b in product(get_factorizations(divisor), get_factorizations(quotient)):
                factorizations.add((*a, *b))
                factorizations.add((*b, *a))

        divisor += 1
    
    res = []

    for f in factorizations:
        res.append(list(f))


    return res

def get_all_factorizations(num):
    combo_factors = get_factorizations(num)
    combo_factors2 = [ [1] + l for l in combo_factors]
    combo_factors3 = [ l + [1] for l in combo_factors]
    combo_factors4 = [ l + [1] for l in combo_factors2]


    combos = combo_factors + combo_factors2 + combo_factors3 + combo_factors4
    
    return combos

In [14]:
# ---------- Helper Functions -----------

import itertools
import copy 
# this function returns a list of all the prime factors of num
# including duplicates
def find_prime_factors(num):
    factors = []
    divisor = 2
    while divisor <= num:
        if num % divisor == 0:
            factors.append(divisor)
            num = num // divisor  # Integer division
        else:
            divisor += 1
    return factors


def all_combinations(numbers):
    return [list(perm) for perm in set(itertools.permutations(numbers))]

directions = {'up':[-1,0],
              'down': [1,0],
              'left': [0,-1],
              'right': [0,1]}
direction_char = {'up':'|',
              'down': '|',
              'left': '-',
              'right': '-'}
opposite_directions = {'up':'down',
                       'down': 'up',
                       'left':'right',
                       'right':'left'}
moves = ['up','down','left','right']

def opposite_move(move):
    return opposite_directions[move]

def update_position(pos,move,dist):
    i = pos[0]
    j = pos[1]
    a = directions[move][0]
    b = directions[move][1]
    return [i + dist*a,j + dist*b]

def check_valid_last_move(pos,next_d,dist,n,m,k):
    # pos - current position on the board
    # next_d - next direction to move
    # dist - distance to travel 
    # n - dimension of the grid
    # m - number of steps to take 
    # k - current step number
    i = pos[0]
    j = pos[1]
    a = directions[next_d][0]
    b = directions[next_d][1]
    end_i = i + (dist)*a
    end_j = j + (dist)*b
    
    last_move = (k == m-1) # last step to take
    # we end in a valid last position
    valid_last_position = (end_i == -1 or end_i == n or end_j == -1 or end_j == n)

    return last_move and valid_last_position

def check_valid_move(pos,next_d,dist,n,m,k):
    if check_valid_last_move(pos,next_d,dist,n,m,k):
        return True
    i = pos[0]
    j = pos[1]
    a = directions[next_d][0]
    b = directions[next_d][1]
    end_i = i + (dist)*a
    end_j = j + (dist)*b
    
    if (end_i >= n) or (end_i < 0):
        return False
    if (end_j >= n) or (end_j < 0):
        return False

    return True

def check_compatibility(mirror, direction):
    return True
backward_mirror_valid_directions = {'up': 'left',
                                'down': 'right',
                                'right': 'down',
                                'left': 'up'}
forward_mirror_valid_directions ={'up': 'right',
                                  'down': 'left',
                                  'right':'up',
                                  'left': 'down'} 

def update2(grid,pos,next_d,dist, n, m, k):
    # print(f'Valid Last Move: {check_valid_last_move(pos,next_d,dist,n,m, k)}')
    is_valid_last_move = check_valid_last_move(pos,next_d,dist,n,m, k)
    if is_valid_last_move:
        dist = dist - 1
    if not check_valid_move(pos,next_d,dist,n,m,k):
        return grid
    
    i = pos[0]
    j = pos[1]
    c = direction_char[next_d]
    a = directions[next_d][0]
    b = directions[next_d][1]
    
    next_i = 0
    next_j = 0
    
    flag = False
    valid_directions = set()
    
    for k2 in range(dist):
        next_i = i + (k2+1)*a
        next_j = j + (k2+1)*b
        
        temp_c = grid[next_i][next_j]
        is_mirror = (temp_c == '\\' or temp_c == '/')
        
        if is_mirror and k2 != dist-1:
            flag = True
            return grid, flag, valid_directions
        
        if k2 == dist - 1 and is_mirror:
            if is_valid_last_move: 
                flag = True
                return grid, flag, valid_directions 
            if temp_c == '\\':
                valid_directions.add(backward_mirror_valid_directions[next_d])
            if temp_c == '/':
                valid_directions.add(forward_mirror_valid_directions[next_d])

        if k2 == dist-1 and temp_c != 'X' and (not is_valid_last_move):
            flag == True
            return grid, flag, valid_directions
        if temp_c != 'X':
            grid[next_i][next_j] = '+'
        else : grid[next_i][next_j] = c

    if len(valid_directions) == 0:
        valid_directions = {'left','right','up','down'}
    
    return grid,flag,valid_directions

def update(grid,pos,next_d,dist, n, m, k):
    # print(f'Valid Last Move: {check_valid_last_move(pos,next_d,dist,n,m, k)}')
    if check_valid_last_move(pos,next_d,dist,n,m, k):
        dist = dist - 1
    if not check_valid_move(pos,next_d,dist,n,m,k):
        return grid
    
    i = pos[0]
    j = pos[1]
    c = direction_char[next_d]
    a = directions[next_d][0]
    b = directions[next_d][1]
    
    next_i = 0
    next_j = 0
    
    for k2 in range(dist):
        next_i = i + (k2+1)*a
        next_j = j + (k2+1)*b
        grid[next_i][next_j] = c
    
    return grid

def forward_or_back(prev_d,move):
    forward_slash = ((prev_d == 'up' and move == 'right') 
                    or(prev_d == 'left' and move == 'down')
                    or (prev_d == 'right' and move == 'up')
                    or (prev_d == 'down' and move == 'left'))
    if forward_slash: return '/'
    back_slash = ((prev_d == 'up' and move == 'left') or
                  (prev_d == 'right' and move == 'down') or
                  (prev_d == 'left' and move == 'up') or
                  (prev_d == 'down' and move == 'right')
                 ) 
    if back_slash: return '\\'
    
    return 'M'

In [5]:
def build_grid(m,n,top = [],left = [],bottom = [],right = []):
    rows = ['X' for _ in range(m)]
    grid = [rows.copy() for _ in range(n)]
    
    return grid


def print_grid(grid):
    for row in grid:
        print('[ ' + ' '.join(row) + ' ]')

In [28]:
MIRROR_CHARS = {'/', '\\'}

def is_orthogonally_adjacent(grid):
    for i, row in enumerate(grid[:-1]):
        for j, symbol in enumerate(row[:-1]):
            if symbol in MIRROR_CHARS and (grid[i+1][j] in MIRROR_CHARS or grid[i][j+1] in MIRROR_CHARS):
                return True
    return False

def overlap_grids(grid_1: list[list[str]], grid_2: list[list[str]]) -> list[list[str]]:
    # Construct new grid, if possible
    new_grid: list[list[str]] = []

    for row_1, row_2 in zip(grid_1, grid_2):
        new_grid.append([])
        for symbol_1, symbol_2 in zip(row_1, row_2):
            if symbol_1 == symbol_2:
                new_grid[-1].append(symbol_1)
            elif symbol_1 in {'-','|'} and symbol_2 == '+':
                new_grid[-1].append('+')
            elif symbol_2 in {'-','|'} and symbol_1 == '+':
                new_grid[-1].append('+')
            elif symbol_1 == 'X':
                new_grid[-1].append(symbol_2)
            elif symbol_2 == 'X':
                new_grid[-1].append(symbol_1)
            elif {symbol_1, symbol_2}.issubset({'-', '|','+'}):
                new_grid[-1].append('+')
            else:
                raise ValueError('Invalid grids')
            
    # Check for newly orthogonally adjacent mirrors
    # Loop over all but last row and column, check (i+1, j) and (i, j+1)
    if is_orthogonally_adjacent(new_grid):
        raise ValueError('Invalid Grid')
            
    return new_grid



from itertools import product
from functools import reduce

def overlap_many_grids(list_of_list_of_grids: list[list[list[list[str]]]], merged_grids):
    # sort so that shorter lists of grids appear sooner
    list_of_list_of_grids = sorted(list_of_list_of_grids, key=lambda x: len(x))

    for grid_combo in product(*list_of_list_of_grids):
        try:
            new_grid = reduce(overlap_grids, grid_combo)
            merged_grids.append(copy.deepcopy(new_grid))
            # print('Valid Merged Grid:')
            # print_grid(new_grid)
            # print()
        except:
            pass


In [29]:
# dynamic programming solution here
def dp(grid_dp,pos,k, prev_d, next_d,path,n,m,sum, prod, target, valid_grids,flag2 = True):
    # m is the number of steps in the path 
    # k is the number of steps taken
    # n is the dimension of the n x n grid
    dist = path[k]
    
    if k == m-1:
        if not check_valid_last_move(pos,next_d,dist,n,m,k) : return
    else:
        if not check_valid_move(pos,next_d,dist,n,m,k): return

    # print(f'Previous Move: {prev_d}')
    # print(f'Previous Sum: {sum}')
    # print(f'Previous Prod: {prod}')
    sum += dist
    if not (prev_d == next_d):
        prod = prod * sum
        sum = 0
    # print('Before Update')
    # print_grid(grid_dp)
    flag = False
    valid_next_moves = {'left','right','up','down'}
    grid_dp2,flag,valid_next_moves = update2(copy.deepcopy(grid_dp),pos,next_d,dist, n, m, k)
    if is_orthogonally_adjacent(grid_dp2): return
    if flag: return
    # print('After Update')
    # print_grid(grid_dp2)
    # # update our position
    # print(f'Position Before: {pos}')
    pos = update_position(pos,next_d,dist)
    # print(f'Position After: {pos}')

    # after this point next_d is our previous direction
    prev_d = next_d

    # we took a step so increment our number of steps taken
    # if we've taken all the possible steps, append and return
    k = k+1
    if k == m:
        # print(f'Prod is: {prod}')
        # print(f'Target is: {target}')
        # print(f'Sum is: {sum}')
        
        if prod == target:
            if flag2:
                valid_grids.append(copy.deepcopy(grid_dp2))
            else:
                valid_grids.append([copy.deepcopy(grid_dp2),path])
        return
    dist = path[k]
    
    # otherwise we keep building on our path
    # we make sure in this loop to not enter paths that
    # are illegal
    is_last_move = (k == m-1)
    for move in moves:
        if move == opposite_move(prev_d) or prev_d == move: continue
        if not check_valid_move(pos,move,dist,n,m,k): continue
        valid_last_move = check_valid_last_move(pos,move,dist,n,m,k)
        if is_last_move and (not valid_last_move): continue
        if move not in valid_next_moves: continue



        # this is a valid move we can make so lets explore it
        # print(f'Next Move: {prev_d}')
        # print(f'Next Sum: {sum}')
        # print(f'Next Prod: {prod}')

        temp_c = grid_dp2[pos[0]][pos[1]]
        
        grid_dp2[pos[0]][pos[1]] = forward_or_back(prev_d,move)

        dp(grid_dp2,pos,k,prev_d, move,path,n,m,sum,prod,target,valid_grids)
        grid_dp2[pos[0]][pos[1]] = temp_c

    return

In [11]:


target = 225
combos = get_all_factorizations(target)


In [16]:
n = 10
grid = build_grid(n,n)
# print_grid(grid)

valid_grids = []
for path in combos:
    pos = [8,-1]
    k = 0
    next_d = 'right'
    sum = 0
    prod = 1
    m = len(path)
    dp(copy.deepcopy(grid),pos,k,'',next_d,path,n,m,sum,prod,target,valid_grids)

In [ ]:
# Print Valid Grids for example
i = 1
for grid in valid_grids:
    print(f'Valid Grid Number {i}')
    # print(f'Path: {grid[1]}')
    i += 1
    print_grid(grid)
    print()

In [19]:
# Indices are as follows:
# 0 - target
# 1 - initial position
# 2 - list of posible paths
# 3 - initial direction
starting_positions = []
# Left Side
# 27
starting_positions.append([27,[3,-1],get_all_factorizations(27),'right'])
# 12
starting_positions.append([12,[7,-1],get_all_factorizations(12),'right'])
# 225
starting_positions.append([225,[8,-1],get_all_factorizations(225),'right'])

# Bottom Side
# 2025
starting_positions.append([2025,[10,0],get_all_factorizations(2025),'up'])
# 12
starting_positions.append([12,[10,3],get_all_factorizations(12),'up'])
# 64
starting_positions.append([64,[10,4],get_all_factorizations(64),'up'])
# 5
starting_positions.append([5,[10,5],get_all_factorizations(5),'up'])
# 405
starting_positions.append([405,[10,7],get_all_factorizations(405),'up'])


# Right Side
# 4
starting_positions.append([4,[1,10],get_all_factorizations(4),'left'])
# 27
starting_positions.append([27,[2,10],get_all_factorizations(27),'left'])
# 16
starting_positions.append([16,[6,10],get_all_factorizations(16),'left'])

# Top Side
# 112
starting_positions.append([112,[-1,2],get_all_factorizations(112),'down'])
# 48
starting_positions.append([48,[-1,4],get_all_factorizations(48),'down'])
# 3087
starting_positions.append([3087,[-1,5],get_all_factorizations(3087),'down'])
# 9
starting_positions.append([9,[-1,6],get_all_factorizations(9),'down'])
# 1
starting_positions.append([1,[-1,9],get_all_factorizations(1),'down'])




In [25]:
# Indices are as follows for initial_state:
# 0 - target
# 1 - initial position
# 2 - list of posible paths
# 3 - initial direction

list_of_list_of_grids = []

for initial_state in starting_positions:
    target = initial_state[0]
    pos = initial_state[1]
    combos = initial_state[2]
    next_d = initial_state[3]
    n = 10
    grid = build_grid(n,n)

    valid_grids = []
    for path in combos:
        k = 0
        sum = 0
        prod = 1
        m = len(path)
        dp(copy.deepcopy(grid),pos,k,'',next_d,path,n,m,sum,prod,target,valid_grids)
    list_of_list_of_grids.append(valid_grids)








In [26]:
for e in list_of_list_of_grids:
    print(len(e))

4
6
14
34
7
54
2
15
2
2
4
39
74
4
1
2


In [37]:
test_list = []
test_list.append(list_of_list_of_grids[0])
test_list.append(list_of_list_of_grids[1])
test_list.append(list_of_list_of_grids[2])
merged_grids1 = []
overlap_many_grids(test_list, merged_grids1)

In [31]:
test_list = []
merged_grids2 = []
test_list.append(list_of_list_of_grids[3])
test_list.append(list_of_list_of_grids[4])
test_list.append(list_of_list_of_grids[5])
test_list.append(list_of_list_of_grids[6])
test_list.append(list_of_list_of_grids[7])
overlap_many_grids(test_list,merged_grids2)

In [33]:
test_list = []
merged_grids3 = []
test_list.append(list_of_list_of_grids[8])
test_list.append(list_of_list_of_grids[9])
test_list.append(list_of_list_of_grids[10])
overlap_many_grids(test_list,merged_grids3)

In [35]:
test_list = []
merged_grids4 = []
test_list.append(list_of_list_of_grids[11])
test_list.append(list_of_list_of_grids[12])
test_list.append(list_of_list_of_grids[13])
test_list.append(list_of_list_of_grids[14])
test_list.append(list_of_list_of_grids[15])

overlap_many_grids(test_list,merged_grids4)


In [38]:
# 1 - left
# 2 - bottom
# 3 - right
# 4 - top
print(len(merged_grids1))
print(len(merged_grids2))
print(len(merged_grids3))
print(len(merged_grids4))

123
490
9
84


In [41]:
# merge bottom (2) with right (3)
list_of_list_of_grids_bottom_and_right = []
list_of_list_of_grids_bottom_and_right.append(copy.deepcopy(merged_grids2))
list_of_list_of_grids_bottom_and_right.append(copy.deepcopy(merged_grids3))

In [43]:
test_list = []
merged_bottom_and_right  = []
overlap_many_grids(list_of_list_of_grids_bottom_and_right, merged_bottom_and_right)

In [44]:
print(len(merged_bottom_and_right))

82


In [45]:
# merge left (1) with top (4)
list_of_list_of_grids_left_and_top = []
list_of_list_of_grids_left_and_top.append(copy.deepcopy(merged_grids1))
list_of_list_of_grids_left_and_top.append(copy.deepcopy(merged_grids4))

In [46]:
test_list = []
merged_left_and_top = []
overlap_many_grids(list_of_list_of_grids_left_and_top,merged_left_and_top)

In [47]:
print(len(merged_bottom_and_right))
print(len(merged_left_and_top))

82
72


In [50]:
# merge  merged_left_and_top with merged_bottom_and_right
test_list = []
merged_all = []
test_list.append(copy.deepcopy(merged_left_and_top))
test_list.append(copy.deepcopy(merged_bottom_and_right))
overlap_many_grids(test_list,merged_all)

In [51]:
print_grid(merged_all[0])

[ - - \ - + + \ + + \ ]
[ - - + - + + + + \ + ]
[ / - + - / + + / + + ]
[ + - / - + + + + + \ ]
[ + - + - + + / + + + ]
[ | / + - \ + + \ + + ]
[ / | \ - + / + + / + ]
[ - + - \ | | | | | | ]
[ - + - + / + / + / | ]
[ / + - + + / + + - + ]
